In [50]:
import pandas as pd


In [51]:
df=pd.read_csv("../input/fake-news/train.csv")

In [52]:
df=df.dropna()

In [53]:
X=df.drop('label',axis=1)

In [54]:
Y=df['label']

In [55]:
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.preprocessing.text import one_hot
from keras.layers import Bidirectional

In [56]:
voc_size=5000

In [57]:
Train=X.copy()

In [58]:
Train.reset_index(inplace=True)

In [59]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [60]:
ps=PorterStemmer()
corpus=[]
for i in range(0, len(Train)):
    review=re.sub('[^a-zA-Z]', ' ', Train['title'][i])
    review=review.lower()
    review=review.split()
    
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [61]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]

sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
embedded_docs

In [62]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ...,  845, 1573, 2051],
       [   0,    0,    0, ..., 4131, 4109,  593],
       [   0,    0,    0, ..., 2511, 3420, 1393],
       ...,
       [   0,    0,    0, ..., 1803, 4986, 4349],
       [   0,    0,    0, ..., 2044, 2018, 3455],
       [   0,    0,    0, ..., 3086, 3651, 2139]], dtype=int32)

In [63]:
from keras.layers import Dropout

In [64]:
embedding_vect_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vect_features, input_length = sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               112800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [65]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(Y)

In [66]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)


In [81]:
y_test.shape

(6035,)

In [67]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 8s 42ms/step - loss: 0.3011 - accuracy: 0.8540 - val_loss: 0.1922 - val_accuracy: 0.9191
Epoch 2/10
192/192 [==============================] - 7s 36ms/step - loss: 0.1360 - accuracy: 0.9495 - val_loss: 0.1987 - val_accuracy: 0.9233
Epoch 3/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0934 - accuracy: 0.9689 - val_loss: 0.2141 - val_accuracy: 0.9193
Epoch 4/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0637 - accuracy: 0.9788 - val_loss: 0.2794 - val_accuracy: 0.9226
Epoch 5/10
192/192 [==============================] - 7s 36ms/step - loss: 0.0444 - accuracy: 0.9851 - val_loss: 0.3075 - val_accuracy: 0.9213
Epoch 6/10
192/192 [==============================] - 8s 40ms/step - loss: 0.0283 - accuracy: 0.9903 - val_loss: 0.3377 - val_accuracy: 0.9168
Epoch 7/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0245 - accuracy: 0.9923 - val_loss: 0.3993 - val_accuracy: 0.9201

In [68]:
Test=pd.read_csv("../input/fake-news/test.csv")

In [69]:
Test_id=Test['id']

In [70]:
# here we are removing these columns as they are not so important
Test=Test.drop(['text','author','id'],axis=1)


In [82]:
Test.shape

(5200, 1)

In [71]:
Test.fillna("fake fake fake", inplace=True)

In [72]:
ps=PorterStemmer()
corpus_test=[]
for i in range(0,len(Test)):
    review = re.sub('[^a-zA-Z]', " ",Test['title'][i] )
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = " ".join(review)
    corpus_test.append(review)

In [73]:
onehot_Test = [one_hot(words,voc_size) for words in corpus_test]

In [74]:
sentence_length = 20
embedded_docs_test = pad_sequences(onehot_Test, padding='pre', maxlen=sentence_length)
print(embedded_docs_test)

[[   0    0    0 ... 1803 4986 4349]
 [   0    0    0 ... 4775  819 4892]
 [   0    0    0 ... 4809 2305 4707]
 ...
 [   0    0    0 ... 1803 4986 4349]
 [   0    0    0 ... 2725 1396  284]
 [   0    0    0 ... 1803 4986 4349]]


In [83]:
len(embedded_docs)

18285

In [75]:
x_test=np.array(embedded_docs_test)

In [79]:
x_test.shape

(5200, 20)

In [76]:
y_pred = model.predict_classes(x_test)

In [80]:
y_pred.shape

(5200, 1)

In [84]:
val=[]
for i in predicted_value:
    val.append(i[0])


In [85]:
submission = pd.DataFrame({'id':Test_id, 'label':val})
submission.shape

(5200, 2)

In [ ]:
submission.to_csv('submission.csv',index=False)